## CS 4662: Group Project
#### MoodReader: Facial Emotion Detection with Deep Learning
#### Group: Alvin Lew, Ly Jacky Nhiayi, Fabio Carrasco, Diane Tabilas, Denise Tabilas

### 1. Reading Data
- Read data retrieved from Kaggle (https://github.com/matiassingers/awesome-readme)

In [40]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib
import tensorflow
import keras

In [41]:
# this is how you read an image
import matplotlib.image as mpimg
mpimg.imread('./train/angry/im0.png').flatten()

array([0.27450982, 0.3137255 , 0.32156864, ..., 0.41568628, 0.42745098,
       0.32156864], dtype=float32)

In [42]:
import os 
test_train = ["train"]
emotions = ["angry", "disgusted", "fearful", "happy", "neutral", "sad", "surprised"]

for tt in test_train:
    count = 0

    # iterate through each emotion folder containing images
    for e in emotions:
        path = f'./{tt}/{e}'
        
        # count the number of files inside the folder
        file_count = len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])
        count += file_count
        print(f'  {e}: {file_count}')

    print(f'files in {tt}: {count}')
    print()


  angry: 3995
  disgusted: 436
  fearful: 4097
  happy: 7215
  neutral: 4965
  sad: 4830
  surprised: 3171
files in train: 28709


In [43]:
# constants about our data
TRAIN_SIZE = 28709
TEST_SIZE = 7178
IMAGE_LENGTH = 48

DATASET_SIZE = TRAIN_SIZE+TEST_SIZE
IMAGE_PIXELS = IMAGE_LENGTH ** 2

In [44]:
# these are the directories that we read from
test_train = ["train"]
emotions = ["angry", "disgusted", "fearful", "happy", "neutral", "sad", "surprised"]

# initialize empty arrays; data for the pixels and different labels
data = np.empty((DATASET_SIZE, IMAGE_PIXELS))
tt_labels = np.empty((DATASET_SIZE), dtype=object)
emotion_labels = np.empty((DATASET_SIZE), dtype=object)

i = 0
for tt in test_train:
    for emotion in emotions:
        # iterate through all the folders
        directory = f'./{tt}/{emotion}'
        
        for filename in os.listdir(directory):
            fpath = os.path.join(directory, filename)
            
            # flatten the image into 1d and save it to our numpy data array
            data[i] = mpimg.imread(fpath).flatten() 

            # mark the labels of the observation
            tt_labels[i] = tt
            emotion_labels[i] = emotion
            i += 1

In [45]:
raw_data = data

In [46]:
data

array([[0.27450982, 0.3137255 , 0.32156864, ..., 0.41568628, 0.42745098,
        0.32156864],
       [0.59215689, 0.58823532, 0.57647061, ..., 0.75686276, 0.71764708,
        0.72156864],
       [0.45882353, 0.45490196, 0.44313726, ..., 0.18039216, 0.18431373,
        0.19215687],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [47]:
from sklearn import preprocessing

# normalize/scale the data
data = preprocessing.scale(data)
data

array([[-0.29787667, -0.15949672, -0.110313  , ...,  0.19215365,
         0.21894105, -0.11274792],
       [ 0.62188984,  0.65142547,  0.65656543, ...,  1.24599895,
         1.10806883,  1.10363699],
       [ 0.23581496,  0.25754893,  0.25542898, ..., -0.53463619,
        -0.52600381, -0.50628423],
       ...,
       [-1.09273663, -1.08626492, -1.07775963, ..., -1.09184175,
        -1.09072009, -1.090626  ],
       [-1.09273663, -1.08626492, -1.07775963, ..., -1.09184175,
        -1.09072009, -1.090626  ],
       [-1.09273663, -1.08626492, -1.07775963, ..., -1.09184175,
        -1.09072009, -1.090626  ]])

In [48]:
print(tt_labels[:5])
print(emotion_labels[:5])

['train' 'train' 'train' 'train' 'train']
['angry' 'angry' 'angry' 'angry' 'angry']


In [49]:
# compile the numpy arrays into a dataframe
df = pd.DataFrame(data=data)
df['obs_type'] = tt_labels
df['emotion'] = emotion_labels

In [50]:
df

,0,1,2,3,4,5,6,7,8,9,...,2296,2297,2298,2299,2300,2301,2302,2303,obs_type,emotion
0,-0.297877,-0.159497,-0.110313,-0.209068,-0.364243,-0.355061,-0.326600,-0.290838,-0.418958,-0.384891,...,1.216414,0.605536,0.218260,0.338074,0.070847,0.192154,0.218941,-0.112748,train,angry
1,0.621890,0.651425,0.656565,0.790479,0.743287,0.587716,0.325817,0.709769,1.103612,1.144367,...,0.090358,0.251375,0.168286,-0.268572,1.002422,1.245999,1.108069,1.103637,train,angry
2,0.235815,0.257549,0.255429,0.116086,-0.118125,-0.417912,-0.620827,-0.667690,-0.694596,-0.727656,...,-0.613427,-0.570783,-0.568826,-0.528563,-0.529773,-0.534636,-0.526004,-0.506284,train,angry
3,0.712731,0.755687,0.632969,0.838650,0.977099,0.939686,1.029403,0.982662,1.103612,1.328932,...,0.077562,0.327267,0.455635,0.548543,0.573407,0.604001,0.675520,0.423892,train,angry
4,1.791469,1.856224,1.918965,1.958624,1.813899,1.253944,0.811931,0.917688,-0.090818,-0.556273,...,0.602202,0.175484,0.168286,0.598066,0.818558,0.700907,0.303048,0.221162,train,angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35882,-1.092737,-1.086265,-1.077760,-1.076145,-1.077984,-1.084141,-1.094149,-1.109516,-1.127741,-1.149520,...,-1.125270,-1.114673,-1.106043,-1.098067,-1.093621,-1.091842,-1.090720,-1.090626,None,None
35883,-1.092737,-1.086265,-1.077760,-1.076145,-1.077984,-1.084141,-1.094149,-1.109516,-1.127741,-1.149520,...,-1.125270,-1.114673,-1.106043,-1.098067,-1.093621,-1.091842,-1.090720,-1.090626,None,None
35884,-1.092737,-1.086265,-1.077760,-1.076145,-1.077984,-1.084141,-1.094149,-1.109516,-1.127741,-1.149520,...,-1.125270,-1.114673,-1.106043,-1.098067,-1.093621,-1.091842,-1.090720,-1.090626,None,None
35885,-1.092737,-1.086265,-1.077760,-1.076145,-1.077984,-1.084141,-1.094149,-1.109516,-1.127741,-1.149520,...,-1.125270,-1.114673,-1.106043,-1.098067,-1.093621,-1.091842,-1.090720,-1.090626,None,None


In [59]:
# here are the two final datasets that we will be working with
test = df[df['obs_type'] == 'test']
train = df[df['obs_type'] == 'train']

train

,0,1,2,3,4,5,6,7,8,9,...,2296,2297,2298,2299,2300,2301,2302,2303,obs_type,emotion
0,-0.297877,-0.159497,-0.110313,-0.209068,-0.364243,-0.355061,-0.326600,-0.290838,-0.418958,-0.384891,...,1.216414,0.605536,0.218260,0.338074,0.070847,0.192154,0.218941,-0.112748,train,angry
1,0.621890,0.651425,0.656565,0.790479,0.743287,0.587716,0.325817,0.709769,1.103612,1.144367,...,0.090358,0.251375,0.168286,-0.268572,1.002422,1.245999,1.108069,1.103637,train,angry
2,0.235815,0.257549,0.255429,0.116086,-0.118125,-0.417912,-0.620827,-0.667690,-0.694596,-0.727656,...,-0.613427,-0.570783,-0.568826,-0.528563,-0.529773,-0.534636,-0.526004,-0.506284,train,angry
3,0.712731,0.755687,0.632969,0.838650,0.977099,0.939686,1.029403,0.982662,1.103612,1.328932,...,0.077562,0.327267,0.455635,0.548543,0.573407,0.604001,0.675520,0.423892,train,angry
4,1.791469,1.856224,1.918965,1.958624,1.813899,1.253944,0.811931,0.917688,-0.090818,-0.556273,...,0.602202,0.175484,0.168286,0.598066,0.818558,0.700907,0.303048,0.221162,train,angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28704,1.053385,0.767271,0.621171,0.597795,0.571005,0.612856,0.236269,-0.472766,-0.510838,-0.714472,...,1.318783,1.301209,1.280201,1.266614,1.247573,1.233886,1.204191,1.187114,train,surprised
28705,0.553759,1.022133,1.423444,0.597795,-0.831866,-0.908156,-0.940639,-0.940583,-0.615842,-0.569457,...,1.433948,1.288560,1.080307,0.870437,0.720498,0.264833,-0.033379,0.471594,train,surprised
28706,-0.979185,-0.900911,-0.853595,-0.835290,-0.831866,-0.895586,-0.851092,-0.680685,-0.077693,-0.002577,...,1.382763,1.339154,1.280201,1.179950,1.063709,1.015849,0.819703,0.662399,train,surprised
28707,0.508338,0.628256,0.762749,0.886821,1.026323,1.115670,1.195706,1.281545,1.313622,1.328932,...,1.779442,1.731261,1.692484,1.675172,1.639815,1.585168,1.528602,1.509099,train,surprised


### 2. Data Preprocessing
- Separate features and labels on our datasets 
- Create a subset of the data with PCA

In [52]:
feature_cols = [x for x in range(48*48)]

X_train =  train[feature_cols]
X_test = test[feature_cols]
y_train = train["emotion"]
y_test = test["emotion"]

In [53]:
y_train.shape

(28709,)

In [54]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,-0.297877,-0.159497,-0.110313,-0.209068,-0.364243,-0.355061,-0.326600,-0.290838,-0.418958,-0.384891,...,0.921590,1.212197,1.216414,0.605536,0.218260,0.338074,0.070847,0.192154,0.218941,-0.112748
1,0.621890,0.651425,0.656565,0.790479,0.743287,0.587716,0.325817,0.709769,1.103612,1.144367,...,0.216168,0.256862,0.090358,0.251375,0.168286,-0.268572,1.002422,1.245999,1.108069,1.103637
2,0.235815,0.257549,0.255429,0.116086,-0.118125,-0.417912,-0.620827,-0.667690,-0.694596,-0.727656,...,-0.789710,-0.698473,-0.613427,-0.570783,-0.568826,-0.528563,-0.529773,-0.534636,-0.526004,-0.506284
3,0.712731,0.755687,0.632969,0.838650,0.977099,0.939686,1.029403,0.982662,1.103612,1.328932,...,-0.188796,-0.052976,0.077562,0.327267,0.455635,0.548543,0.573407,0.604001,0.675520,0.423892
4,1.791469,1.856224,1.918965,1.958624,1.813899,1.253944,0.811931,0.917688,-0.090818,-0.556273,...,2.175673,1.676954,0.602202,0.175484,0.168286,0.598066,0.818558,0.700907,0.303048,0.221162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28704,1.053385,0.767271,0.621171,0.597795,0.571005,0.612856,0.236269,-0.472766,-0.510838,-0.714472,...,1.339618,1.328386,1.318783,1.301209,1.280201,1.266614,1.247573,1.233886,1.204191,1.187114
28705,0.553759,1.022133,1.423444,0.597795,-0.831866,-0.908156,-0.940639,-0.940583,-0.615842,-0.569457,...,1.653138,1.547855,1.433948,1.288560,1.080307,0.870437,0.720498,0.264833,-0.033379,0.471594
28706,-0.979185,-0.900911,-0.853595,-0.835290,-0.831866,-0.895586,-0.851092,-0.680685,-0.077693,-0.002577,...,1.444125,1.405846,1.382763,1.339154,1.280201,1.179950,1.063709,1.015849,0.819703,0.662399
28707,0.508338,0.628256,0.762749,0.886821,1.026323,1.115670,1.195706,1.281545,1.313622,1.328932,...,1.849089,1.831873,1.779442,1.731261,1.692484,1.675172,1.639815,1.585168,1.528602,1.509099


In [55]:
from  sklearn.decomposition  import  PCA

k = 100
pca = PCA(n_components=k)

# new datasets for after PCA
X_train_pca = pca.fit_transform(X_train)

### 3.1 SVM Model

In [56]:
from sklearn.svm import SVC

svm = SVC(C=1, kernel='rbf', gamma=0.0005, random_state=1)
svm = svm.fit(X_train_pca, y_train)

In [57]:
svm

SVC(C=1, gamma=0.0005, random_state=1)

In [58]:
y_predict = svm.predict(X_test_pca)

NameError: name 'X_test_pca' is not defined

In [ ]:
from sklearn.metrics import accuracy_score

def print_accuracy(test_labels, predict_labels, model_name=''):
    print(f'{model_name} model accuracy:')
    print(accuracy_score(test_labels, predict_labels))

In [ ]:
print_accuracy(y_test, y_predict, 'SVM')

In [ ]:
# pickle

### 3.2 Regular ANN

In [ ]:
from keras.src.applications.densenet import layers
from keras import models

input_size = 2304
hidden_neurons = 500
out_size = 7

model_ann = models.Sequential()
model_ann.add(layers.Dense(hidden_neurons, input_dim = input_size))
model_ann.add(layers.Activation('sigmoid'))

model_ann.add(layers.Dense(hidden_neurons, input_dim = hidden_neurons))
model_ann.add(layers.Activation('sigmoid'))

model_ann.add(layers.Dense(hidden_neurons, input_dim = hidden_neurons))
model_ann.add(layers.Activation('sigmoid'))

model_ann.add(layers.Dense(out_size, input_dim = hidden_neurons))
model_ann.add(layers.Activation('softmax'))


In [ ]:
model_ann.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

In [ ]:
X_train.shape

In [ ]:
y_train_encoded.shape

In [ ]:
from keras.utils import np_utils
y_trainANN = y_train_encoded.values.argmax(1)
y_testANN = y_test_encoded.values.argmax(1)

In [ ]:
y_trainANN.shape

In [ ]:
y_trainANN = np_utils.to_categorical(y_trainANN, 7)
y_testANN = np_utils.to_categorical(y_testANN, 7)

In [ ]:
y_trainANN.shape

In [ ]:
X_train.shape

In [ ]:
fitted_model = model_ann.fit(X_train, y_trainANN, validation_split=0.33, batch_size=32, epochs=15, verbose=1)

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(fitted_model.history['accuracy'], label='accuracy')
plt.plot(fitted_model.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
X_test_ANN = test[feature_cols]
y_predict_ann = model_ann.predict(X_test_ANN, verbose=1)
# print (y_predict_ann.shape)

In [ ]:
score = model_ann.evaluate(X_test_ANN, y_testANN, verbose=1)
print('The accuracy is: ', score[1])

### 3.3 Convolution Neural Network (CNN)

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam

In [ ]:
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(IMAGE_LENGTH, IMAGE_LENGTH, 1)))

model_cnn.add(layers.MaxPooling2D((2, 2)))
model_cnn.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(48, activation='relu'))
model_cnn.add(layers.Dense(7)) 

In [ ]:
model_cnn.summary()

In [ ]:
X_train.values

In [ ]:
X_train.values.shape

In [ ]:
# we need to reshape this for keras
X_train_cnn = X_train.values.reshape(TRAIN_SIZE, 48, 48, 1)
X_test_cnn = X_test.values.reshape(TEST_SIZE, 48, 48, 1)

X_train_cnn

In [ ]:
# we need to encode this and reshape it as well
y_train

In [ ]:
y_train_encoded = pd.get_dummies(y_train)
y_test_encoded = pd.get_dummies(y_test)

y_train_encoded

In [ ]:
y_train_encoded.values.argmax(1)

In [ ]:
y_train_cnn = y_train_encoded.values.argmax(1).reshape(TRAIN_SIZE, 1)
y_test_cnn = y_test_encoded.values.argmax(1).reshape(TEST_SIZE, 1)

y_train_cnn

In [ ]:
print(X_train_cnn.shape)
print(y_train_cnn.shape)
print(X_test_cnn.shape)
print(y_test_cnn.shape)

In [ ]:
model_cnn.summary()

In [ ]:
model_cnn.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history_cnn = model_cnn.fit(X_train_cnn, y_train_cnn, epochs=10, 
                    validation_data=(X_test_cnn, y_test_cnn))

In [ ]:
model_cnn.save("models/CNN")

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(history_cnn.history['accuracy'], label='accuracy')
plt.plot(history_cnn.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
test_loss, test_acc = model_cnn.evaluate(X_test_cnn, y_test_cnn, verbose=2)

In [ ]:
import pickle
with open('./histories/cnnHistory.pickle', 'wb') as file_pi:
    pickle.dump(history_cnn.history, file_pi)
with open('./histories/cnnHistory.pickle', 'rb') as file_pi:
    pickled_history = pickle.load(file_pi)

### 3.4 LeNet 5 (with data augmentation)

In [ ]:
# LeNet5 Model
model_LeNet = keras.Sequential()

model_LeNet.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model_LeNet.add(layers.AveragePooling2D())

model_LeNet.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model_LeNet.add(layers.AveragePooling2D())

model_LeNet.add(layers.Flatten())

model_LeNet.add(layers.Dense(units=120, activation='relu'))
model_LeNet.add(layers.Dense(units=84, activation='relu'))
model_LeNet.add(layers.Dense(units=10, activation = 'softmax'))

In [ ]:
model_LeNet.summary()

In [ ]:
model_LeNet.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history_LeNet = model_LeNet.fit(X_train_cnn, y_train_cnn, epochs=10, 
                    validation_data=(X_test_cnn, y_test_cnn))

In [ ]:

with open('./histories/history_LeNet.pickle', 'wb') as file_pi:
    pickle.dump(history_LeNet.history, file_pi)
with open('./histories/history_LeNet.pickle', 'rb') as file_pi:
    pickled_history_LeNet = pickle.load(file_pi)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(history_LeNet.history['accuracy'], label='Training Accuracy')
plt.plot(history_LeNet.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
# LeNet5 Model with data augmentation (flip and rotation)
model_Le_and_Augment = keras.Sequential()
  
model_Le_and_Augment.add(layers.RandomFlip("horizontal"))
model_Le_and_Augment.add(layers.RandomRotation(0.2))

model_Le_and_Augment.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model_Le_and_Augment.add(layers.AveragePooling2D())

model_Le_and_Augment.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model_Le_and_Augment.add(layers.AveragePooling2D())

model_Le_and_Augment.add(layers.Flatten())

model_Le_and_Augment.add(layers.Dense(units=120, activation='relu'))
model_Le_and_Augment.add(layers.Dense(units=84, activation='relu'))
model_Le_and_Augment.add(layers.Dense(units=10, activation = 'softmax'))

In [ ]:
model_Le_and_Augment.compile(optimizer=Adam(learning_rate=0.002),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history_Le_and_Augment = model_Le_and_Augment.fit(X_train_cnn, y_train_cnn, epochs=15, 
                    validation_data=(X_test_cnn, y_test_cnn))

In [ ]:
with open('./histories/history_Le_and_Augment.pickle', 'wb') as file_pi:
    pickle.dump(history_Le_and_Augment.history, file_pi)
with open('./histories/history_Le_and_Augment.pickle', 'rb') as file_pi:
    pickled_history_Le_and_Augment = pickle.load(file_pi)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(history_Le_and_Augment.history['accuracy'], label='Training Accuracy')
plt.plot(history_Le_and_Augment.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
# LeNet5 Model with data augmentation (only rotation)
model_Le_Augment_rot = keras.Sequential()
  
model_Le_Augment_rot.add(layers.RandomRotation(0.2))

model_Le_Augment_rot.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model_Le_Augment_rot.add(layers.AveragePooling2D())

model_Le_Augment_rot.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model_Le_Augment_rot.add(layers.AveragePooling2D())

model_Le_Augment_rot.add(layers.Flatten())

model_Le_Augment_rot.add(layers.Dense(units=120, activation='relu'))
model_Le_Augment_rot.add(layers.Dense(units=84, activation='relu'))
model_Le_Augment_rot.add(layers.Dense(units=10, activation = 'softmax'))

In [ ]:
model_Le_Augment_rot.compile(optimizer=Adam(learning_rate=0.003),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history_Le_Augment_rot = model_Le_Augment_rot.fit(X_train_cnn, y_train_cnn, epochs=25, 
                    validation_data=(X_test_cnn, y_test_cnn))

In [ ]:
with open('./histories/history_Le_Augment_rot.pickle', 'wb') as file_pi:
    pickle.dump(history_Le_Augment_rot.history, file_pi)
with open('./histories/history_Le_Augment_rot.pickle', 'rb') as file_pi:
    pickle_history_Le_Augment_rot = pickle.load(file_pi)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(history_Le_Augment_rot.history['accuracy'], label='Training Accuracy')
plt.plot(history_Le_Augment_rot.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

### 3.5 VGG-16

In [ ]:
X_train_vgg16 = []
X_test_vgg16 = []
# need to resize to use the VGG16 from keras
# since our data is black and white, we copy the pixel values 3 times
for img in X_train_cnn:
    X_train_vgg16.append(np.repeat(img,3,axis = -1))
for img in X_test_cnn:
    X_test_vgg16.append(np.repeat(img,3,axis = -1))
X_train_vgg16 = np.array(X_train_vgg16)
X_test_vgg16 = np.array(X_test_vgg16)

In [ ]:
X_train_vgg16.shape

In [ ]:
X_test_vgg16.shape

In [ ]:
y_train.shape

In [ ]:
from keras.applications import VGG16

# Load the pre-trained VGG16 model
model_16 = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the layers of the pre-trained model so that they are not trained again during fine-tuning
for layer in model_16.layers:
    layer.trainable = False

new_model = models.Sequential()
new_model.add(model_16)
# new_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same'))
new_model.add(layers.Flatten())
new_model.add(layers.Dense(256, activation='relu'))
new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(7, activation='softmax'))

# Compile the model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
X_train_vgg16.shape

In [ ]:
from keras.utils import to_categorical

# Reshaping y_train and y_test for vgg16
y_train_vgg16 = to_categorical(y_train_cnn, num_classes=7)
y_test_vgg16 = to_categorical(y_test_cnn, num_classes=7)

In [ ]:
y_test_cnn.shape

In [ ]:
history_vgg16 = new_model.fit(X_train_vgg16, y_train_vgg16, epochs=10, 
                    validation_data=(X_test_vgg16, y_test_vgg16))

In [ ]:
with open('./histories/history_vgg16.pickle', 'wb') as file_pi:
    pickle.dump(history_vgg16.history, file_pi)
with open('./histories/history_vgg16.pickle', 'rb') as file_pi:
    pickle_history_vgg16 = pickle.load(file_pi)

In [ ]:

fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(history_vgg16.history['accuracy'], label='accuracy')
plt.plot(history_vgg16.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
test_loss, test_acc = new_model.evaluate(X_test_vgg16, y_test_vgg16, verbose=2)

### 3.6 MobileNetV2

In [ ]:
raw_data_df = pd.DataFrame(data=raw_data)
raw_data_df['obs_type'] = tt_labels
raw_data_df['emotion'] = emotion_labels
raw_test = raw_data_df[raw_data_df['obs_type'] == 'test']
raw_train = raw_data_df[raw_data_df['obs_type'] == 'train']
X_train_raw = raw_train[feature_cols]
X_test_raw = raw_test[feature_cols]
y_train_raw = raw_train["emotion"]
y_test_raw = raw_test["emotion"]

In [ ]:
# we need to modify the dataset a bit for this model
X_train_cnn_raw = X_train_raw.values.reshape(TRAIN_SIZE, 48, 48, 1)
X_test_cnn_raw = X_test_raw.values.reshape(TEST_SIZE, 48, 48, 1)

# encode categories
y_train_encoded_raw = pd.get_dummies(y_train_raw)
y_test_encoded_raw = pd.get_dummies(y_test_raw)

y_train_cnn_raw = y_train_encoded_raw.values.argmax(1).reshape(TRAIN_SIZE, 1)
y_test_cnn_raw = y_test_encoded_raw.values.argmax(1).reshape(TEST_SIZE, 1)

In [ ]:
mobilenet_model = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_LENGTH, IMAGE_LENGTH, 3),
                                                    include_top=False,
                                                    weights='imagenet')
mobilenet_model

In [ ]:
# freeze the convolutional layers
mobilenet_model.trainable = False

In [ ]:
mobilenet_model.summary()

In [ ]:
inputs = tf.keras.Input(shape=(48, 48, 1))

x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)

# convert into rgb formatted image for MobileNet to accept it
x = keras.layers.Conv2D(3, (3, 3), padding='same')(inputs)

# start with base model
x = mobilenet_model(x, training=False)  

# add a classification head 
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)

# outputs have 7 possibilities
outputs = tf.keras.layers.Dense(7)(x)

model_mobilenet = tf.keras.Model(inputs, outputs)

In [ ]:
# Settings for the model

# learning rate for training the non-CNN layers
# we'll divide this by 10 when retraining some of the MobileNetV2's CNN layers
BASE_MNET_LR = 0.0003

# epochs to train at first
MNET_INIT_EPOCHS = 10

# epochs to train for fine-tuning and retraining the CNN part of the model
MNET_FINE_TUNE_EPOCHS = 10

MNET_TOTAL_EPOCHS = MNET_INIT_EPOCHS + MNET_FINE_TUNE_EPOCHS

In [ ]:
model_mobilenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=BASE_MNET_LR),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model_mobilenet.summary()

In [ ]:
print(X_train_cnn_raw.shape)
print(X_test_cnn_raw.shape)
print(y_train_encoded_raw.shape)
print(y_test_encoded_raw.shape)

In [ ]:

history_mobile_net = model_mobilenet.fit(X_train_cnn_raw, y_train_encoded_raw, epochs=10, 
                    validation_data=(X_test_cnn_raw, y_test_encoded_raw))
                    

In [ ]:
#model.save("MobileNetV2_TransferLearn_Raw10")

In [ ]:
#model = keras.models.load_model("MobileNetV2_TransferLearn_Raw10")

In [ ]:
with open('./histories/history_mobile_net.pickle', 'wb') as file_pi:
    pickle.dump(history_mobile_net.history, file_pi)
with open('./histories/history_mobile_net.pickle', 'rb') as file_pi:
    pickle_history_mobile_net = pickle.load(file_pi)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(history_mobile_net.history['accuracy'], label='Training Accuracy')
plt.plot(history_mobile_net.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
# Let's try some fine-tuning of the model
mobilenet_model.trainable = True

In [ ]:
print("Current MobileNet layer count: ", len(mobilenet_model.layers))

In [ ]:
# freeze first 100 layers and retrain the last 54
for layer in mobilenet_model.layers[:100]:
    layer.trainable = False

In [ ]:
model_mobilenet.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=BASE_MNET_LR/10),
              metrics=['accuracy'])

In [ ]:
model_mobilenet.summary()

In [ ]:
len(model_mobilenet.trainable_variables)

In [ ]:
# continue to train the model
history_fine_tune = model_mobilenet.fit(X_train_cnn_raw, y_train_encoded_raw,
                              epochs=MNET_TOTAL_EPOCHS,
                              initial_epoch=history_mobile_net.epoch[-1],
                              validation_data=(X_test_cnn_raw, y_test_encoded_raw))

In [ ]:
model_mobilenet.save("models/MobileNetV2")

In [ ]:

acc = history_mobile_net.history['accuracy']
val_acc = history_mobile_net.history['val_accuracy']

acc += history_fine_tune.history['accuracy']
val_acc += history_fine_tune.history['val_accuracy']



In [ ]:
with open('./histories/history_fine_tune.pickle', 'wb') as file_pi:
    pickle.dump(history_fine_tune.history, file_pi)
with open('./histories/history_fine_tune.pickle', 'rb') as file_pi:
    pickle_history_fine_tune = pickle.load(file_pi)

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.plot([10,10],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

In [ ]:
test_loss, test_acc = model_mobilenet.evaluate(X_test_cnn_raw, y_test_encoded_raw , verbose=2)

### 4. Results

SVM (with PCA): 37.67%

CNN (custom standard model): 50.06%

ANN (no PCA): 30.86%

VGG-16 (transfer learning): 43.37%

MobileNetV2 (transfer learning + fine tuning): 46.80%

LeNet5 (RandomRotation(0.2) Layer): 47.78%

In [ ]:
fig = plt.figure(figsize=(4,4), dpi=250)
plt.plot(history_vgg16.history['val_accuracy'], label = 'VGG16 val_accuracy')
plt.plot(history_Le_Augment_rot.history['val_accuracy'], label = 'LeNet5+Augment+Rotation val_accuracy')
plt.plot(history_Le_and_Augment.history['val_accuracy'], label = 'LeNet5+Augment val_accuracy')
plt.plot(history_mobile_net.history['val_accuracy'], label = 'Mobile_net Fine Tuned val_accuracy')
plt.plot(fitted_model.history['val_accuracy'], label = ' ANN val_accuracy')
plt.plot(history_cnn.history['val_accuracy'], label = 'CNN val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='upper right', fontsize='x-small')